#응집도 기반 키워드 추출

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
class Word:
#단어 리스트
#key: 단어 길이(리뷰에서 공백을 기준으로 분리된 토큰)
#word: 실제 단어
#cnt: key와 동일한 단어의 출현 빈도수
#freq: key가 포함된 (key로 시작하는) 단어의 출현 빈도수
    def __init__(self, word):
        self.key = len(word)
        self.word = word
        self.cnt = 1
        self.freq = 1

In [ ]:
class Node:
#id: 노드 아이디
#pid: 부모 노드의 아이디
#dep: 트리에서 깊이
#w : word 클래스 연결
#coh: 응집도 점수
    def __init__(self, id, word):
        self.id = id
        self.pid = -1
        self.dep = 1
        self.w = word
        self.coh = 0

In [ ]:
#word 클래스 sort 해야해서 heap으로 함!
#heap_sort 사용 예정
class Heap: 
    def __init__(self, L = []):
        self.A = L     
        self.make_heap()
    
    def __str__(self):
        return str(self.A)
    def __len__(self):
        return len(self.A)
    
    def heapify_down(self,k,n):
        while 2*k+1 < n :
            L, R = 2*k+1, 2*k+2
            if L < n and self.A[L].key > self.A[k].key : m = L
            else: m = k
            if R < n and self.A[R].key > self.A[m].key : m = R
            #m = A[k],A[L],A[K] 중 최대의 인덱스

            if m != k: #A[k] 가 최소가 아니라면 힙 성질 위배
                self.A[k], self.A[m] = self.A[m],self.A[k]
                k = m 
            else:
                break
    
    def make_heap(self):
        n = len(self.A)
        for k in range(n-1,-1,-1):
            self.heapify_down(k,n)

    def heap_sort(self):
        n = len(self.A)
        for k in range(len(self.A)-1, -1, -1):
            self.A[0], self.A[k] = self.A[k], self.A[0]
            n = n-1
            self.heapify_down(0,n)
    def copy(self):
        return self.A

In [ ]:
def binarySearch(list, item, left,right):
    if left > right:
        return False
    else:
        mid = (left + right)//2
        if item == A[mid].w.word:
            return True
        elif item < A[mid].w.key:
            return binarySearch(A,item,left,mid-1)
        else:
            return binarySearch(A,item,mid+1,right)

In [ ]:
#응집도 분석할 단어 리스트 추출 및 초기화

def getWordList(wdset):
    wordlist = []
    for i in range(len(wdset)):
        if len(wdset[i]) < 1:
            continue
        found = 0
        for a in range(len(wordlist)):
            if wdset[i] == wordlist[a].word:
                wordlist[a].cnt += 1
                wordlist[a].freq += 1
                found = 1
                break

        if found:
            continue
        
        new_word = Word(wdset[i])
        wordlist.append(new_word)

    return wordlist

In [ ]:
#응집도 점수 분석 함수

def getScoredList(list):
    #heap으로 단어 길이 순으로 정렬
    heap = Heap(list)
    heap.heap_sort()
    print("heap: ", len(heap))

    score = [] #응집도,빈도를 가지는 node list
    data = heap.copy()
    
    for i in range(len(data)):
        new_node = Node(i,data[i])
        score.append(new_node)
    print('score: ',len(score))

    
    for i in range(len(score)):
        for j in range(i+1, len(score)):
            a = score[i].w.word
            b = score[j].w.word
            if type(a) == float or type(b) == float:
                print(a)
                print(b)
            if (b[:len(a)+1] == a ):
                score[j].pid = score[i].id
                score[j].dep = score[i].dep + 1
                score[i].w.freq += score[j].w.cnt
                if b == a:
                    score[j].w.word = np.nan

    
    #응집도 점수 계산
    for i in range(len(score)):
        score[i].coh = pow(score[i].w.freq/score[0].w.freq, 1/(score[i].dep+1))

    return score


#블로그 텍스트 데이터를 사용한 키워드 추출

In [ ]:
#데이터 
DATA_PATH = '/content/drive/MyDrive/데이터캠퍼스/심화프로젝트/'
pre_data = pd.read_csv(DATA_PATH +'blog_prep.csv',encoding= 'utf-8')
pre_data.head()

,Unnamed: 0,prep_s,prep_w
0,0,여행 낭만의 도시에서의 맛집이제서야 꺼내보는 기록. 에서는 매우 짧은 2박...,여행 낭만 도시 맛집 이제 기록. 매우 일정 폭염 날씨 이유 여행 싸움 흙흙 경험 ...
1,1,지금까지 포스팅된 여행정보를 보려면 왼쪽 상단에 있는 작은 사진을 클릭해서 전체글 ...,지금 포스팅 여행 정보 왼쪽 상단 사진 클릭 전체 해외 배낭여행 정보 클릭 현재 남...
2,2,코로나 유럽 여행 → 포르투갈 무료 항원 검사 백신 의무화 발표 유럽 백신여...,코로나 유럽 여행 포르투갈 무료 항원 검사 백신 의무 발표 유럽 백신 여권 등록 방...
3,3,안녕하세요 세쿨이입니다 오늘은 여행 이야기 여행에서 빼놓을 수 없는 미술관오...,오늘 여행 이야기 여행 미술관 미술관 정보 정리 시간 미술관 입장료 운영 시간 기본...
4,4,2020년 1월 에서 한달살기 스토리1월 14일 화요일 세번째이야기 1월 14일 ...,살기 스토리 화요일 세번 이야기 화요일 생제르맹 지역 생제르맹데프레 성당 바로 레두...


In [ ]:
prep_w = pre_data['prep_w']
prep_s = pre_data['prep_s']


In [ ]:
#문장으로 테스트 해보기
x = ''
for p in prep_w:
    x += p +' '
x.replace('.','')

test_data = x.split(' ')

#함수 호출하여 점수 계산
print("words: ", len(test_data))
test_s = getWordList(test_data)
temp_s = getScoredList(test_s)

#계산 결과 데이터 프레임화
coh_score =[]
word = []
freq = []

for i in temp_s:
    coh_score.append(i.coh)
    word.append(i.w.word)
    freq.append(i.w.freq)

Coh_sentences = pd.DataFrame({'word': word,'freq':freq,'score': coh_score})
Coh_sentences.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
Coh_sentences.sort_values(by = 'score',ascending = False, inplace = True)
Coh_sentences.head(50)


words:  14587
heap:  4169
score:  4169


,word,freq,score
1087,여행,331,1.495489
1504,사진,168,1.065427
2772,에펠탑,149,1.003373
0,.,148,1.000000
1638,사람,128,0.929981
2425,미술관,95,0.801182
3094,박물관,95,0.801182
1478,유럽,94,0.796954
3065,루브르,85,0.757842
1828,성당,84,0.753371


In [ ]:
#정렬 후 저장
Coh_sentences.sort_values(by = 'score',ascending = False, inplace = True)
Coh_sentences.head(50)
Coh_sentences.to_csv('블로그_응집도_단어.csv', encoding = 'utf-8-sig')

In [ ]:
#60개 post를 한 문자열로 합침
word_total = ''
for i in prep_w:
    word_total += i
word_total = word_total.replace('.','')
word_total = word_total.split(' ')

In [ ]:
#함수 호출하여 점수 계산
print("words: ", len(word_total))
test_w = getWordList(word_total)
temp_w = getScoredList(test_w)

words:  101207
heap:  12043
score:  12043


In [ ]:
#계산 결과 데이터 프레임화
coh_score =[]
word = []
freq = []

for i in temp_s:
    coh_score.append(i.coh)
    word.append(i.w.word)
    freq.append(i.w.freq)

Coh_words = pd.DataFrame({'word': word,'freq':freq,'score': coh_score})
Coh_words.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)




,word,freq,score
0,.,148,1.000000
1,리스,2,0.116248
2,선정,1,0.082199
3,외정,1,0.082199
4,해시,1,0.082199
...,...,...,...
4164,설렜더랬다.,1,0.082199
4165,생폴드방스.,3,0.142374
4166,어니언수프.,1,0.082199
4167,스테인드글라스,3,0.142374


In [ ]:
Coh_words

In [ ]:
#정렬 후 저장
Coh_words.sort_values(by = 'score',ascending = False, inplace = True)
Coh_words.head(50)
Coh_words.to_csv('블로그_응집도_단어.csv', encoding = 'utf-8-sig')

In [ ]:
brunch_keywords = Coh_words['word']
print(brunch_keywords[:20])

4592       사람
5728       우리
2785       시간
3062       여행
2088       생각
         ... 
3473       사이
1806       음식
10739    레스토랑
4992       경우
7457       이동
Name: word, Length: 100, dtype: object


#여행 서적 텍스트 데이터를 이용한 키워드 추출

In [ ]:
#가이드북으로 테스트 해보기
book = open(DATA_PATH + '20개년 20개국.txt', 'r', encoding = 'euc-kr')
book_data= guide.read()
temp = book_data.split('\n')
book_line = ''
for t in temp:
    guide_line += t + ' '


x = ''
for p in prep_s:
    x += p +' '

test_data = x.split(' ')
#함수 호출하여 점수 계산
print("words: ", len(test_data))
test_s = getWordList(test_data)
temp_s = getScoredList(test_s)

#계산 결과 데이터 프레임화
coh_score =[]
word = []
freq = []

for i in temp_w:
    coh_score.append(i.coh)
    word.append(i.w.word)
    freq.append(i.w.freq)

Coh_sentences = pd.DataFrame({'word': word,'freq':freq,'score': coh_score})
Coh_sentences.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
Coh_sentences.sort_values(by = 'score',ascending = False, inplace = True)
Coh_sentences.head(50)


In [ ]:
br = pd.read_csv(DATA_PATH + 'prep_brunch_df.csv',encoding = 'utf-8')
prep = br['sentence_prep']

file = open("brunch.txt", 'w', encoding='utf-8')
for item in prep:
    file.write(item +'\n')

